# Bhattacharyya distance
Here, we compare two PDFs or a field of PDFs using the [Bhattacharyya distance](https://en.wikipedia.org/wiki/Bhattacharyya_distance).

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import utils

# 1D

Let $X$ ~ $N(\mu_x, \sigma_x^2)$ and $Y$ ~ $N(\mu_y, \sigma_y^2)$ be two random variables, of a normal distribution $p(x)$ and $q(x)$. We can compare their Bhattacharyya distance as followed:
$$
    D_B(p, q) = - \ln\left( BC(p,q) \right) \\
    BC(p, q) = \sum\limits_{x \in X} \sqrt{ p(x) q(x) }
$$
For nornal distributions, the closed form can be used:
$$
    D_B(p, q) = \frac{1}{4} \ln\left[ \frac{1}{4} \left( \frac{\sigma_x^2}{\sigma_y^2} + \frac{\sigma_y^2}{\sigma_x^2} + 2 \right) \right] + \frac{1}{4}\left( \frac{(\mu_x - \mu_y)^2}{\sigma_x^2 + \sigma_y^2} \right)
$$
In order to take covariances into account, the following form can be used:
$$
    D_B = \frac{1}{8} ( \mu_x - \mu_y)^T \Sigma^{-1} (\mu_x - \mu_y) + \frac{1}{2} \ln \left( \frac{\det \Sigma}{ \sqrt{ \det \Sigma_x \det \Sigma_y } } \right) \\
    \Sigma = \frac{ \Sigma_x + \Sigma_y }{2}
$$

In [2]:
def bhattacharyya(mu1, var1, mu2, var2):
    return 0.25 * np.log(
        0.25 * (
            var1/var2 + var2/var1 + 2
        )
    ) + 0.25 * (
        (mu1 - mu2)**2 / ( var1 + var2 )
    )

def bhattacharyya_matrix(mu1, sigma1, mu2, sigma2):
    sigma = (sigma1 + sigma2) * 0.5
    return 0.125 * np.transpose((mu1 - mu2).flatten()) @ np.linalg.inv(sigma) @ (mu1 - mu2).flatten()
    + 0.5 * np.log( np.linalg.det(sigma) / np.sqrt( np.linalg.det(sigma1) * np.linalg.det(sigma2) ) )

In [3]:
def normal_dist(x , mean , var):
    return 1 / np.sqrt(2*np.pi * var) * np.exp(-0.5*(x-mean)**2/var)

In [4]:
max_mu = 6.0
max_sigma = 2.0

def show_bhattacharyya_mu(mu_x = 0.0, var_x = 0.1, mu_y = 0.0, var_y = 0.6, max_mu=max_mu, max_sigma=max_sigma):
    mu1 = np.linspace(0, max_mu)
    mu2 = np.linspace(0, max_mu)
    
    mu11, mu22 = np.meshgrid(mu1, mu2)
    
    BD_mu = bhattacharyya(mu11, var_x, mu22, var_y)
    print(bhattacharyya(mu_x, var_x, mu_y, var_y))
    
    plt.figure(figsize=(7,6))
    plt.title(r'$\mu_x$-$\mu_y$-plane')
    plt.imshow(BD_mu)
    plt.colorbar()
    plt.scatter(mu_x/max_mu * (len(mu1)-1), mu_y/max_mu * (len(mu2)-1), c='magenta')
    plt.xlabel(r'$\mu_x$')
    plt.ylabel(r'$\mu_y$')
    plt.xticks(np.linspace(0, len(mu1), 5), np.linspace(mu1[0], mu1[-1], 5))
    plt.yticks(np.linspace(0, len(mu2), 5), np.linspace(mu2[0], mu2[-1], 5))
    plt.xlim(0,len(mu1)-1)
    plt.ylim(0,len(mu2)-1)
    
    plt.show()
    
def show_bhattacharyya_sigma(mu_x = 0.0, var_x = 0.1, mu_y = 0.0, var_y = 0.6, max_mu=max_mu, max_sigma=max_sigma):
    var1 = np.linspace(0.1, max_sigma)
    var2 = np.linspace(0.1, max_sigma)
    
    var11, var22 = np.meshgrid(var1, var2)
    
    BD_var = bhattacharyya(mu_x, var11, mu_y, var22)
    print(bhattacharyya(mu_x, var_x, mu_y, var_y))
    
    plt.figure(figsize=(7,6))
    plt.title(r'$\sigma_x$-$\sigma_y$-plane')
    plt.imshow(BD_var)
    plt.colorbar()
    plt.scatter(var_x / max_sigma * (len(var1)-1), var_y / max_sigma * (len(var2)-1), c='magenta')
    plt.xlabel(r'$\sigma_x$')
    plt.ylabel(r'$\sigma_y$')
    plt.xticks(np.linspace(0, len(var1), 5), np.linspace(var1[0], var1[-1], 5))
    plt.yticks(np.linspace(0, len(var2), 5), np.linspace(var2[0], var2[-1], 5))
    plt.xlim(0,len(var1)-1)
    plt.ylim(0,len(var2)-1)
    
def show_pdfs(mu_x = 0.0, var_x = 0.1, mu_y = 0.0, var_y = 0.6, max_mu=max_mu):
    space = np.linspace(0, max_mu, 200)
    
    x = normal_dist(space, mu_x, var_x)
    y = normal_dist(space, mu_y, var_y)
    
    plt.figure(figsize=(18,6))
    plt.plot(space, x)
    plt.plot(space, y)
    plt.xlim(0,max_mu)
    plt.ylim(0,4)
    plt.show()
    
mu_x_widget = widgets.FloatSlider(
    value=0,
    min=0,
    max=max_mu,
    step=0.1,
    description=r'$\mu_x$:',
    layout=widgets.Layout(left='70px', width='395px')
)

mu_y_widget = widgets.FloatSlider(
    value=0,
    min=0,
    max=max_mu,
    step=0.1,
    orientation='vertical',
    description=r'$\mu_y$:',
    layout=widgets.Layout(left='0px', height='355px')
)

var_x_widget = widgets.FloatSlider(
    value=0.1,
    min=0.1,
    max=max_sigma,
    step=0.1,
    description=r'$\sigma_x$:',
    layout=widgets.Layout(left='70px', width='405px')
)

var_y_widget = widgets.FloatSlider(
    value=0.1,
    min=0.1,
    max=max_sigma,
    step=0.1,
    orientation='vertical',
    description=r'$\sigma_y$:',
    layout=widgets.Layout(top='0px', height='355px')
)

sigma_interact = widgets.interactive_output(show_bhattacharyya_sigma, {
    'mu_x': mu_x_widget,
    'var_x': var_x_widget,
    'mu_y': mu_y_widget,
    'var_y': var_y_widget
})

mu_interact = widgets.interactive_output(show_bhattacharyya_mu, {
    'mu_x': mu_x_widget,
    'var_x': var_x_widget,
    'mu_y': mu_y_widget,
    'var_y': var_y_widget
})

pdf_interact = widgets.interactive_output(show_pdfs, {
    'mu_x': mu_x_widget,
    'var_x': var_x_widget,
    'mu_y': mu_y_widget,
    'var_y': var_y_widget
})

layout = widgets.HBox([
    widgets.VBox([
        mu_x_widget,
        widgets.HBox([
            mu_y_widget,
            mu_interact
        ])
    ]),
    widgets.VBox([
        var_x_widget,
        widgets.HBox([
            var_y_widget,
            sigma_interact
        ])
    ])
])

display(widgets.VBox([
    layout,
    pdf_interact
]))

# 2D (without covariance)

In [5]:
width = 10
percent = 5
 
input_range = 8649.794 # max - min value of sinogram
var = np.ones((64,64)) * input_range * percent*1e-2
Sigma_in = utils.build_covariance_matrix(var, function=utils.exponential, width=width)

algs = ['ramp', 'cosine', 'hamming', 'hann', 'shepp-logan']

@widgets.interact(algo1=algs, algo2=algs)
def compare(algo1='ramp', algo2='cosine'):
    mu1 = np.load('../../Data/iradon/mu_'+algo1+'.npy')
    df_dx1 = np.abs(np.load('../../Data/iradon/df_dx_'+algo1+'.npy'))
    Sigma1 = df_dx1 @ Sigma_in @ np.transpose(df_dx1)
    variance1 = np.reshape(np.abs(np.diagonal(Sigma1)), mu1.shape)
    
    mu2 = np.load('../../Data/iradon/mu_'+algo2+'.npy')
    df_dx2 = np.abs(np.load('../../Data/iradon/df_dx_'+algo2+'.npy'))
    Sigma2 = df_dx2 @ Sigma_in @ np.transpose(df_dx2)
    variance2 = np.reshape(np.abs(np.diagonal(Sigma2)), mu2.shape)
    
    BD = bhattacharyya(mu1, variance1, mu2, variance2)
    
    plt.figure(figsize=(18,6))
    plt.suptitle(algo1)
    plt.subplot(1,2,1)
    plt.imshow(mu1)
    plt.colorbar()

    plt.subplot(1,2,2)
    plt.imshow(variance1)
    plt.colorbar()
    
    plt.show()
    
    plt.figure(figsize=(18,6))
    plt.suptitle(algo2)
    plt.subplot(1,2,1)
    plt.imshow(mu2)
    plt.colorbar()

    plt.subplot(1,2,2)
    plt.imshow(variance2)
    plt.colorbar()
    
    plt.show()
    
    plt.figure(figsize=(18,6))
    plt.suptitle('difference')
    plt.subplot(1,2,1)
    plt.imshow(BD)
    plt.colorbar()
    
    plt.show()

interactive(children=(Dropdown(description='algo1', options=('ramp', 'cosine', 'hamming', 'hann', 'shepp-logan…

# 2D with covariance

In [6]:
width = 10
percent = 5
 
input_range = 8649.794 # max - min value of sinogram
var = np.ones((64,64)) * input_range * percent*1e-2
Sigma_in = utils.build_covariance_matrix(var, function=utils.exponential, width=width)

algs = ['ramp', 'cosine', 'hamming', 'hann', 'shepp-logan']

@widgets.interact(algo1=algs, algo2=algs)
def compare(algo1='ramp', algo2='cosine'):
    mu1 = np.load('../../Data/iradon/mu_'+algo1+'.npy')
    df_dx1 = np.abs(np.load('../../Data/iradon/df_dx_'+algo1+'.npy'))
    Sigma1 = np.abs(df_dx1 @ Sigma_in @ np.transpose(df_dx1))
    
    mu2 = np.load('../../Data/iradon/mu_'+algo2+'.npy')
    df_dx2 = np.abs(np.load('../../Data/iradon/df_dx_'+algo2+'.npy'))
    Sigma2 = np.abs(df_dx2 @ Sigma_in @ np.transpose(df_dx2))
    
    BD = bhattacharyya_matrix(mu1, Sigma1, mu2, Sigma2)
    
    plt.figure(figsize=(18,6))
    plt.suptitle(algo1)
    plt.subplot(1,2,1)
    plt.imshow(mu1)
    plt.colorbar()

    plt.subplot(1,2,2)
    plt.imshow(Sigma1)
    plt.colorbar()
    
    plt.show()
    
    plt.figure(figsize=(18,6))
    plt.suptitle(algo2)
    plt.subplot(1,2,1)
    plt.imshow(mu2)
    plt.colorbar()

    plt.subplot(1,2,2)
    plt.imshow(Sigma2)
    plt.colorbar()
    
    plt.show()
    
    display(widgets.HTML('<h1>Difference: {:.02f}</h1>'.format(BD)))

interactive(children=(Dropdown(description='algo1', options=('ramp', 'cosine', 'hamming', 'hann', 'shepp-logan…